Loading in the packages that we will use later.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import ElasticNetCV

Reading in the data. We force the types of MSSubClass, OverallQual, and OverallCond to be read in as strings even though they would naturally be read in as numbers because the numbers are referencing certain categories.

In [2]:
train = pd.read_csv('train.csv', dtype={'MSSubClass':str, 'OverallQual':str, 'OverallCond':str})
test = pd.read_csv('test.csv', dtype={'MSSubClass':str, 'OverallQual':str, 'OverallCond':str})
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


Check to see which columns contain NaN values. Note that some of the NaNs mean none as denoted by the data description file.

In [3]:
train.columns[train.isna().any()].tolist()

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

NaNs for Alley, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PoolQC, Fence, and MiscFeature all indicate that these elements are not in the house. We can change the NaNs for these features to be read as 'None' instead to avoid confusion with missing values.

In [4]:
train['Alley'].fillna('None', inplace=True)
train['BsmtQual'].fillna('None', inplace=True)
train['BsmtCond'].fillna('None', inplace=True)
train['BsmtExposure'].fillna('None', inplace=True)
train['BsmtFinType1'].fillna('None', inplace=True)
train['BsmtFinType2'].fillna('None', inplace=True)
train['FireplaceQu'].fillna('None', inplace=True)
train['GarageType'].fillna('None', inplace=True)
train['GarageFinish'].fillna('None', inplace=True)
train['GarageQual'].fillna('None', inplace=True)
train['GarageCond'].fillna('None', inplace=True)
train['PoolQC'].fillna('None', inplace=True)
train['Fence'].fillna('None', inplace=True)
train['MiscFeature'].fillna('None', inplace=True)
test['Alley'].fillna('None', inplace=True)
test['BsmtQual'].fillna('None', inplace=True)
test['BsmtCond'].fillna('None', inplace=True)
test['BsmtExposure'].fillna('None', inplace=True)
test['BsmtFinType1'].fillna('None', inplace=True)
test['BsmtFinType2'].fillna('None', inplace=True)
test['FireplaceQu'].fillna('None', inplace=True)
test['GarageType'].fillna('None', inplace=True)
test['GarageFinish'].fillna('None', inplace=True)
test['GarageQual'].fillna('None', inplace=True)
test['GarageCond'].fillna('None', inplace=True)
test['PoolQC'].fillna('None', inplace=True)
test['Fence'].fillna('None', inplace=True)
test['MiscFeature'].fillna('None', inplace=True)

Now we have much fewer features that have NaN values. We can work with these ones individually.

In [5]:
missing = train.columns[train.isna().any()].tolist()

Find how many NaNs we have for each column with missing values.

In [6]:
[print(i, sum(train[i].isna())) for i in missing]

LotFrontage 259
MasVnrType 8
MasVnrArea 8
Electrical 1
GarageYrBlt 81


[None, None, None, None, None]

LotFrontage is a quantitative variable that we can try to fill with the mean value. MasVnrType is a categorical variable that includes a 'None' category. We might be able to assume the blanks mean 'None' but we could also replace the NaNs with the most frequent category. MasVnrArea is quantitative but it is possible that the masonry veneer type is none in which case the area would be zero so we can't necessarily fill with the mean. Electrical is a categorical variable that we could fill with the most common category. We can look at the other Garage variables to determine if there is a garage and if there is, we can fill GarageYrBlt with the mean.

Replace NaNs for LotFrontage with the mean. This seems to make sense since most house properties probably connect to the street.

In [7]:
train['LotFrontage'].fillna(np.mean(train['LotFrontage']), inplace=True)
test['LotFrontage'].fillna(np.mean(train['LotFrontage']), inplace=True)

Look at the most common categories for masonry veneer type.

In [8]:
Counter(train['MasVnrType'])

Counter({'BrkCmn': 15, 'BrkFace': 445, 'None': 864, 'Stone': 128, nan: 8})

Most train don't have a masonry veneer type so we will assume that the NaNs are also 'None'.

In [9]:
train['MasVnrType'].fillna('None', inplace=True)
test['MasVnrType'].fillna('None', inplace=True)

We suspect that if the MasVnrType is 'None', the area of the masonry veneer is probably 0. There were 864 train with type none before we added the 8 that had NaN so we will compare that number to the amount of train that have MasVnrArea of 0.

In [10]:
Counter(train['MasVnrArea']).most_common(5)

[(0.0, 861), (180.0, 8), (108.0, 8), (72.0, 8), (120.0, 7)]

We found 861 train with MasVnrArea of 0 which is pretty close to 864. We will assume most of these train to have lined up. So we can assume that the 8 train with MasVnrArea of NaN to really have areas of 0.

In [11]:
train['MasVnrArea'].fillna(0, inplace=True)
test['MasVnrArea'].fillna(0, inplace=True)

There was only one house with an Electrical Value of NaN. We want to look at it and see why.

In [12]:
train[train['Electrical'].isna()==True]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1379,1380,80,RL,73.0,9735,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2008,WD,Normal,167500


The Utilities for this house is AllPub which includes electricity so nothing about the house indicates there is no electricity. We can look at if there is a most common type of electricity and replace this house's electricity with that type.

In [13]:
Counter(train['Electrical'])

Counter({'FuseA': 94,
         'FuseF': 27,
         'FuseP': 3,
         'Mix': 1,
         'SBrkr': 1334,
         nan: 1})

Most train have SBrkr electrical so we will make this house have SBrkr electrical as well.

In [14]:
train['Electrical'].fillna('SBrkr', inplace=True)
test['Electrical'].fillna('SBrkr', inplace=True)

Accounting for missing values for GarageYrBlt when the missing values likely indicate no garage at all seems like more work than it's worth. We can probably safely assume that GarageFinish, GarageQual, and GarageCond will give mostly the same information as GarageYrBlt so we will drop GarageYrBlt.

In [15]:
train.drop(['GarageYrBlt'], axis=1, inplace=True)
test.drop(['GarageYrBlt'], axis=1, inplace=True)

We made sure that the training set doesn't have any missing values. We probably took care of most of the test set as well above, but we need to first make sure before we try and fit a model:

In [16]:
test.columns[test.isna().any()].tolist()

['MSZoning',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'GarageCars',
 'GarageArea',
 'SaleType']

We will replace the missing values for MSZoning, Utilities, Exterior1st, Exterior2nd, KitchenQual, Functional, and SaleType with the most frequently occuring category from the training ste.

In [17]:
test['MSZoning'].fillna(train['MSZoning'].value_counts().index.tolist()[0], inplace=True)
test['Utilities'].fillna(train['Utilities'].value_counts().index.tolist()[0], inplace=True)
test['Exterior1st'].fillna(train['Exterior1st'].value_counts().index.tolist()[0], inplace=True)
test['Exterior2nd'].fillna(train['Exterior2nd'].value_counts().index.tolist()[0], inplace=True)
test['KitchenQual'].fillna(train['KitchenQual'].value_counts().index.tolist()[0], inplace=True)
test['Functional'].fillna(train['Functional'].value_counts().index.tolist()[0], inplace=True)
test['SaleType'].fillna(train['SaleType'].value_counts().index.tolist()[0], inplace=True)

For BsmtFinSF1 and BsmtFinSF2, we can check and see if BsmtFinType1 and BsmtFinType2 are 'None' or 'Unf'. If they are, we will put 0 for these variables. If they're not, we will impute with the mean. For BsmtUnfSF, if both BsmtFinType1 and BsmtFinType2 are 'None', then we will put 0. If not, we will impute with the mean.

In [18]:
print(test[test['BsmtFinSF1'].isna()==True][['BsmtFinType1']])
print(test[test['BsmtFinSF2'].isna()==True][['BsmtFinType2']])
print(test[test['BsmtUnfSF'].isna()==True][['BsmtFinType1', 'BsmtFinType2']])

    BsmtFinType1
660         None
    BsmtFinType2
660         None
    BsmtFinType1 BsmtFinType2
660         None         None


It is 'None' for all of them so we can replace the missing values for each of these with 0.

In [19]:
test['BsmtFinSF1'].fillna(0, inplace=True)
test['BsmtFinSF2'].fillna(0, inplace=True)
test['BsmtUnfSF'].fillna(0, inplace=True)

For TotalBsmtSF, we will replace the missing value with the sum of BsmtFinSF1, BsmtFinSF2, and BsmtUnfSF.

In [20]:
test['TotalBsmtSF'].fillna(test['BsmtFinSF1']+test['BsmtFinSF2']+test['BsmtUnfSF'], inplace=True)

For BsmtFullBath and BsmtHalfBath, if BsmtCond is 'None', then replace them with 0.

In [21]:
print(test[test['BsmtFullBath'].isna()==True][['BsmtCond']])
print(test[test['BsmtHalfBath'].isna()==True][['BsmtCond']])

    BsmtCond
660     None
728     None
    BsmtCond
660     None
728     None


In [22]:
test['BsmtFullBath'].fillna(0, inplace=True)
test['BsmtHalfBath'].fillna(0, inplace=True)

Similarly for GarageCars and GarageArea, we will check if GarageType is 'None'. Then we will replace with 0.

In [23]:
print(test[test['GarageCars'].isna()==True][['GarageType']])
print(test[test['GarageArea'].isna()==True][['GarageType']])

     GarageType
1116     Detchd
     GarageType
1116     Detchd


GarageType isn't 'None' for these so we will replace with the mean of the training set.

In [24]:
test['GarageCars'].fillna(np.mean(train['GarageCars']), inplace=True)
test['GarageArea'].fillna(np.mean(train['GarageArea']), inplace=True)

We will now proceed with trying to fit a model with the data cleaned.

We also want to drop our target variable (SalePrice) before fitting a model.

In [25]:
y_train = train['SalePrice']
train.drop('SalePrice', axis=1, inplace=True)

We also don't want the sale ID to be in the model so we will drop that:

In [26]:
Id = train['Id']
train.drop('Id', axis=1, inplace=True)
Id_test = test['Id']
test.drop('Id', axis=1, inplace=True)

We want to account for categorical variables by creating dummy variables.

We will keep our original data without dummies so we can see what happens if we drop unimportant features later in a different model.

In [27]:
train_orig = train
test_orig = test

In [28]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [29]:
print('Train columns: ', len(train.columns))
print('Test columns: ', len(test.columns))

Train columns:  332
Test columns:  315


One issue we need to fix is that the train and test datasets have a different number of variables because of the different dummy variables used. We must fix that before we model the data.

In [30]:
test_cols = test.columns
train_cols = train.columns
[x for x in test_cols if x not in train_cols]

['MSSubClass_150']

We will drop MSSubClass_150 as a variable from test.

In [31]:
test.drop('MSSubClass_150', axis=1, inplace=True)

We will add the variables unique to train to our test dataset and assign each observation a value of 0 for each of these variables.

In [32]:
test_cols = test.columns
train_cols = train.columns
train_uniq = [x for x in train_cols if x not in test_cols]

In [33]:
for i in train_uniq:
    test[i] = 0

We are now good to go.

Scaling the data:

In [34]:
scaler = StandardScaler()
scaler.fit(train)
X_train = scaler.transform(train)
scaler.fit(test)
X_test = scaler.transform(test)

Cross validation with 3 folds. Doing a random search with various values for the hyperparameters eta0 and tol to find which one creates the linear model that achieves the best negative MSE

In [35]:
param_dist = {"eta0": [ .001, .003, .01, .03, .1, .3, 1, 3],
             "tol": [0.01, 0.001, 0.0001]}

linear_regression_model = SGDRegressor()

n_iter_search = 8
random_search = RandomizedSearchCV(linear_regression_model, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=3, scoring='neg_mean_squared_error')
random_search.fit(X_train, y_train)

print("Best Parameters: {}".format(random_search.best_params_))
print("Best Negative MSE: {}".format(random_search.best_score_))

Best Parameters: {'tol': 0.001, 'eta0': 0.003}
Best Negative MSE: -2103458206.7621574


Getting the best tol and eta0 to use for my linear model.

In [36]:
best_tol = list(random_search.best_params_.values())[0]
best_eta0 = list(random_search.best_params_.values())[1]

Fitting a linear model using the log of SalePrice because we don't want negative prices to be able to be predicted. I take e to the values gotten for predictions to untransform SalePrice. Getting the RMSLE for the model.

In [37]:
linear_regression_model = SGDRegressor(tol=best_tol, eta0=best_eta0)
linear_regression_model.fit(X_train, np.log(y_train))
predictions = np.exp(linear_regression_model.predict(X_train))
msle = mean_squared_error(np.log(y_train), np.log(predictions))
print("RMSLE: {}".format(np.sqrt(msle)))

RMSLE: 0.11802802243943955


Looking at a summary for the predictions and the actual SalePrice to make sure nothing looks too crazy.

In [38]:
temp = pd.Series(predictions)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
temp.describe()

count     1460.000
mean    177887.738
std      76784.735
min      36019.889
25%     127523.247
50%     161943.842
75%     209101.835
max     850595.039
dtype: float64

In [39]:
y_train.describe()

count     1460.000
mean    180921.196
std      79442.503
min      34900.000
25%     129975.000
50%     163000.000
75%     214000.000
max     755000.000
Name: SalePrice, dtype: float64

Now using the model to predict on the test set.

In [40]:
test_preds = np.exp(linear_regression_model.predict(X_test))

Creating a dataframe of the test ids and the SalePrice predicted by our model.

In [41]:
d = {'Id':Id_test,'SalePrice':test_preds}
output = pd.DataFrame(d)

In [42]:
output.head()

,Id,SalePrice
0,1461,105787.838
1,1462,196279.473
2,1463,134332.820
3,1464,156276.073
4,1465,188222.948


Outputting my predictions.

In [43]:
output.to_csv('output.csv', index=False)

Regularizing with Elastic Net Regularization to account fo overfitting. Trying different alpha values.

In [44]:
clf = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas=[.1, 1, 10])
clf.fit(X_train, np.log(y_train))
train_predictions = np.exp(clf.predict(X_train))
msle = mean_squared_error(np.log(y_train), np.log(train_predictions))
print("RMSLE: {}".format(np.sqrt(msle)))

RMSLE: 0.11901160915545013


In [45]:
en_preds = np.exp(clf.predict(X_test))

In [46]:
d = {'Id':Id_test,'SalePrice':en_preds}
output_en = pd.DataFrame(d)

In [47]:
output_en.to_csv('output_en.csv', index=False)

Dropping some probably unimportant variables to try out another model.

In [48]:
train_orig.drop(['LotFrontage', 'Alley', 'LotShape', 'LotConfig', 'MasVnrType', 'GarageType', 'GarageCars'], 
                axis=1, inplace=True)

In [49]:
test_orig.drop(['LotFrontage', 'Alley', 'LotShape', 'LotConfig', 'MasVnrType', 'GarageType', 'GarageCars'], 
                axis=1, inplace=True)

Get dummies for train_orig and test_orig:

In [50]:
train_orig = pd.get_dummies(train_orig)
test_orig = pd.get_dummies(test_orig)

Which test_orig variables are not in train_orig?

In [51]:
test_cols = test_orig.columns
train_cols = train_orig.columns
[x for x in test_cols if x not in train_cols]

['MSSubClass_150']

Drop MSSubClass_150

In [52]:
test_orig.drop('MSSubClass_150', axis=1, inplace=True)

Adding dummy variables unique to train_orig to test_orig

In [53]:
train_uniq = [x for x in train_cols if x not in test_cols]

In [54]:
for i in train_uniq:
    test_orig[i] = 0

Ready to go.

Scaling the data

In [55]:
scaler = StandardScaler()
scaler.fit(train_orig)
X_train = scaler.transform(train_orig)
scaler.fit(test_orig)
X_test = scaler.transform(test_orig)

Elastic Net Regression Again

In [56]:
clf = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas=[.1, 1, 10])
clf.fit(X_train, np.log(y_train))
train_predictions = np.exp(clf.predict(X_train))
msle = mean_squared_error(np.log(y_train), np.log(train_predictions))
print("RMSLE: {}".format(np.sqrt(msle)))

RMSLE: 0.11926192528118189


In [57]:
en_preds = np.exp(clf.predict(X_test))

Creating prediction data frame and outputting

In [58]:
d = {'Id':Id_test,'SalePrice':en_preds}
output_en = pd.DataFrame(d)

In [59]:
output_en.to_csv('output_en_2.csv', index=False)